In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import os

import tensorflow as tf

from tensorflow.keras import layers

In [ ]:
print(tf. __version__)

2.15.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from zipfile import ZipFile

with ZipFile('/content/drive/MyDrive/Colab Notebooks/Bangkit/dataset v2.zip','r') as zipobj:
  zipobj.extractall('/content/drive/MyDrive/Colab Notebooks/Bangkit/dataset v2')

In [ ]:
# Read the labels file
label_file_path = '/content/drive/MyDrive/Colab Notebooks/Bangkit/dataset v2/train/_classes.csv'

data = pd.read_csv(label_file_path)

# Separate filenames and labels
filenames = data['filename'].tolist()
labels = data.drop(columns=['filename']).values.tolist()
data.head()

In [ ]:
labels_tensor = tf.convert_to_tensor(labels, dtype=tf.int64)

def load_and_preprocess_image(file_path):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0
    return image


file_paths = [f"/content/drive/MyDrive/Colab Notebooks/Bangkit/dataset v2/train/{filename}" for filename in filenames]
image_ds = tf.data.Dataset.from_tensor_slices(file_paths)
image_ds = image_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

label_ds = tf.data.Dataset.from_tensor_slices(labels_tensor)

dataset = tf.data.Dataset.zip((image_ds, label_ds))

BATCH_SIZE = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE

dataset = dataset.shuffle(buffer_size=len(filenames))
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Add the global average pooling layer and output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(len(data.columns) - 1, activation='softmax')(x)

# Combine the base model and custom layers
model = Model(inputs=base_model.input, outputs=predictions)

model.summary()

9406464/9406464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (N

In [ ]:
# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Use 'categorical_crossentropy' for one-hot encoded labels
              metrics=['accuracy'],
              #validation_split=.2
              )

model.fit(dataset, epochs=10)

Epoch 1/10
29/29 [==============================] - 13s 69ms/step - loss: 2.0418 - accuracy: 0.4112
Epoch 2/10
29/29 [==============================] - 5s 48ms/step - loss: 0.8300 - accuracy: 0.7971
Epoch 3/10
29/29 [==============================] - 5s 44ms/step - loss: 0.5373 - accuracy: 0.8622
Epoch 4/10
29/29 [==============================] - 4s 44ms/step - loss: 0.4064 - accuracy: 0.9019
Epoch 5/10
29/29 [==============================] - 6s 48ms/step - loss: 0.3140 - accuracy: 0.9361
Epoch 6/10
29/29 [==============================] - 4s 43ms/step - loss: 0.2561 - accuracy: 0.9526
Epoch 7/10
29/29 [==============================] - 6s 44ms/step - loss: 0.2111 - accuracy: 0.9658
Epoch 8/10
29/29 [==============================] - 5s 45ms/step - loss: 0.1817 - accuracy: 0.9735
Epoch 9/10
29/29 [==============================] - 5s 47ms/step - loss: 0.1553 - accuracy: 0.9746
Epoch 10/10
29/29 [==============================] - 6s 44ms/step - loss: 0.1300 - accuracy: 0.9813


In [ ]:
# Load the test CSV file
test_csv_file_path = '/content/drive/MyDrive/Colab Notebooks/Bangkit/dataset v2/test/_classes.csv'
test_data = pd.read_csv(test_csv_file_path)

test_filenames = test_data['filename'].tolist()
test_labels = test_data.drop(columns=['filename']).values.tolist()

In [ ]:
test_labels_tensor = tf.convert_to_tensor(test_labels, dtype=tf.int64)

def load_and_preprocess_image(file_path):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])  # Resize to match MobileNet's input size
    image = image / 255.0  # Normalize to [0, 1]
    return image

test_file_paths = [f"/content/drive/MyDrive/Colab Notebooks/Bangkit/dataset v2/test/{filename}" for filename in test_filenames]
test_image_ds = tf.data.Dataset.from_tensor_slices(test_file_paths)
test_image_ds = test_image_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels_tensor)
test_dataset = tf.data.Dataset.zip((test_image_ds, test_label_ds))

test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

3/3 [==============================] - 2s 573ms/step - loss: 0.3820 - accuracy: 0.9000
Test Loss: 0.38197624683380127
Test Accuracy: 0.8999999761581421


In [ ]:
predictions = model.predict(test_dataset)
predicted_class = tf.argmax(predictions, axis=1).numpy()

predicted_class

In [ ]:
data.columns[1:]

Index([' Ayam-Goreng', ' Bakso', ' Burger', ' Ketoprak', ' Mie-Goreng',
       ' Mie-basah', ' Nasi', ' Nasi-Goreng', ' Nasi-Padang', ' Nasi-Uduk',
       ' Roti-Putih', ' Sate-Ayam', ' Soto', ' Tahu', ' Tahu Telur',
       ' Telur-Ceplok', ' Tempe-Goreng'],
      dtype='object')

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import balanced_accuracy_score, classification_report

labels_list = data.columns[1:]

enc = LabelEncoder()
enc.fit(labels_list)

test_classes = np.argmax(test_labels, axis=1)

test_classes_enc = enc.inverse_transform(test_classes)
predicted_class_enc = enc.inverse_transform(predicted_class)

print(classification_report(test_classes_enc,predicted_class_enc))

               precision    recall  f1-score   support

  Ayam-Goreng       1.00      1.00      1.00         6
        Bakso       1.00      1.00      1.00        10
       Burger       1.00      0.80      0.89         5
     Ketoprak       1.00      1.00      1.00         4
   Mie-Goreng       1.00      1.00      1.00         4
    Mie-basah       1.00      1.00      1.00         2
         Nasi       1.00      1.00      1.00         1
  Nasi-Goreng       1.00      0.67      0.80         6
  Nasi-Padang       1.00      1.00      1.00         2
    Nasi-Uduk       1.00      0.50      0.67         2
   Roti-Putih       0.90      1.00      0.95         9
    Sate-Ayam       0.89      0.89      0.89        18
         Soto       0.83      0.83      0.83         6
         Tahu       0.83      1.00      0.91         5
   Tahu Telur       0.50      1.00      0.67         1
 Telur-Ceplok       0.57      1.00      0.73         4
 Tempe-Goreng       1.00      0.60      0.75         5

     acc

In [ ]:
model.save('MobileNetV5 with new classes.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
#@keras.saving.register_keras_serializable(package="ComplexModels")
#class BalancedCategoricalAccuracy(tf.keras.metrics.CategoricalAccuracy):
#    def __init__(self, name='balanced_categorical_accuracy', dtype=None):
#        super().__init__(name, dtype=dtype)
#
#    def update_state(self, y_true, y_pred, sample_weight=None):
#        # Convert one-hot encoded y_true to integer labels
#        y_true_int = tf.argmax(y_true, axis=-1)
#
#        # Calculate the inverse class frequency
#        cls_counts = tf.math.bincount(y_true_int)
#        cls_counts = tf.math.reciprocal_no_nan(tf.cast(cls_counts, self.dtype))
#
#        # Assign weights based on the class frequency
#        weight = tf.gather(cls_counts, y_true_int)
#
#        return super().update_state(y_true, y_pred, sample_weight=weight)

In [ ]:
#model.compile(optimizer='adam',
#              loss='categorical_crossentropy',  # Use 'categorical_crossentropy' for one-hot encoded labels
#              metrics=[BalancedCategoricalAccuracy()]) # Use custome balanced accuracy for imbalanced class cases
#
#model.fit(dataset, epochs=10)

Epoch 1/10
29/29 [==============================] - 13s 46ms/step - loss: 0.1398 - balanced_categorical_accuracy: 0.9733
Epoch 2/10
29/29 [==============================] - 4s 47ms/step - loss: 0.0954 - balanced_categorical_accuracy: 0.9915
Epoch 3/10
29/29 [==============================] - 6s 46ms/step - loss: 0.0725 - balanced_categorical_accuracy: 0.9915
Epoch 4/10
29/29 [==============================] - 6s 54ms/step - loss: 0.0582 - balanced_categorical_accuracy: 0.9936
Epoch 5/10
29/29 [==============================] - 4s 45ms/step - loss: 0.0480 - balanced_categorical_accuracy: 0.9956
Epoch 6/10
29/29 [==============================] - 5s 50ms/step - loss: 0.0408 - balanced_categorical_accuracy: 0.9971
Epoch 7/10
29/29 [==============================] - 7s 54ms/step - loss: 0.0355 - balanced_categorical_accuracy: 0.9967
Epoch 8/10
29/29 [==============================] - 7s 64ms/step - loss: 0.0330 - balanced_categorical_accuracy: 0.9950
Epoch 9/10
29/29 [=====================

In [ ]:
#model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_2[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']      

In [ ]:
model.save("model_MobileNetV5.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
